In [1]:
# Construction of dataset

import os, itertools, time, pickle, sys
import subprocess
from xml.dom import minidom
from collections import Counter, OrderedDict
from operator import itemgetter
from nltk.corpus import wordnet
import tensorflow as tf
import tensorflow_hub as hub
from scipy import spatial
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np
import scipy.sparse as sp
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from math import ceil, exp
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import networkx as nx
import matplotlib.pyplot as plt
from nltk.tokenize.treebank import TreebankWordDetokenizer
%matplotlib inline  


USE_folder = "/home/vlead/USE"
alignment_folder = "reference-alignment/"

# Load reference alignments 
def load_alignments(folder):
    alignments = []
    for f in os.listdir(folder):
        doc = minidom.parse(folder + f)
        ls = list(zip(doc.getElementsByTagName('entity1'), doc.getElementsByTagName('entity2')))
        alignments.extend([(a.getAttribute('rdf:resource'), b.getAttribute('rdf:resource')) for (a,b) in ls])
    return alignments
        
reference_alignments = load_alignments(alignment_folder)

In [2]:
flatten = lambda l: [item for sublist in l for item in sublist]

class Ontology():
    def __init__(self, ontology):
        self.ontology = ontology
        self.ontology_obj = minidom.parse(ontology)
        self.root = self.ontology_obj.documentElement
        self.subclasses = self.parse_subclasses()
        self.object_properties = self.parse_object_properties()
        self.data_properties = self.parse_data_properties()
        self.triples = self.parse_triples()
        self.classes = self.parse_classes()
    
    def get_child_node(self, element, tag):
        return [e for e in element._get_childNodes() if type(e)==minidom.Element and e._get_tagName() == tag]
        
    def has_attribute_value(self, element, attribute, value):
        return True if element.getAttribute(attribute).split("#")[-1] == value else False
    
    def get_subclass_triples(self):
        return [(b,a,"subclass_of") for (a,b) in self.get_subclasses()]
    
    def parse_triples(self, union_flag=0, subclass_of=True):
        obj_props = self.object_properties
        data_props = self.data_properties
        props = obj_props + data_props
        all_triples = []
        for prop in props:
            domain_children = self.get_child_node(prop, "rdfs:domain")
            range_children = self.get_child_node(prop, "rdfs:range")
            domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children])
            range_prop = self.filter_null([self.extract_ID(el) for el in range_children])
            if not domain_children or not range_children:
                continue
            if not domain_prop:
                domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children[0].getElementsByTagName("owl:Class")])
            if not range_prop:
                range_prop = self.filter_null([self.extract_ID(el) for el in range_children[0].getElementsByTagName("owl:Class")])
            if domain_prop and range_prop:
                if union_flag == 0:
                    all_triples.extend([(el[0], el[1], self.extract_ID(prop)) for el in list(itertools.product(domain_prop, range_prop))])
                else:
                    all_triples.append(("###".join(domain_prop), "###".join(range_prop), self.extract_ID(prop)))
        if subclass_of:
            all_triples.extend(self.get_subclass_triples())
        return list(set(all_triples))
    
    def get_triples(self, union_flag=0, subclass_of=True, include_inv=True):
        return self.parse_triples(union_flag, subclass_of)

    def parse_subclasses(self, union_flag=0):
        subclasses = self.root.getElementsByTagName("rdfs:subClassOf")
        subclass_pairs = []
        for el in subclasses:
            inline_subclasses = self.extract_ID(el)
            if inline_subclasses:
                subclass_pairs.append((el, el.parentNode))
            else:
                level1_class = self.get_child_node(el, "owl:Class")
                if not level1_class:
                    continue
                if self.extract_ID(level1_class[0]):
                    subclass_pairs.append((level1_class[0], el.parentNode))
                else:
                    level2classes = level1_class[0].getElementsByTagName("owl:Class")
                    
                    subclass_pairs.extend([(elem, el.parentNode) for elem in level2classes if self.extract_ID(elem)])
        return subclass_pairs
        
    def get_subclasses(self):
        return [(self.extract_ID(a), self.extract_ID(b)) for (a,b) in self.subclasses]
    
    def filter_null(self, data):
        return [el for el in data if el]
    
    def extract_ID(self, element):
        element_id = element.getAttribute("rdf:ID") or element.getAttribute("rdf:resource") or element.getAttribute("rdf:about")
        return element_id.split("#")[-1]
    
    def parse_classes(self):
        class_elems = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        subclass_classes = list(set(flatten([el[:-1] for el in self.triples])))
        return list(set(self.filter_null(class_elems + subclass_classes)))
    
    def get_classes(self):
        return self.classes
    
    def get_entities(self):
        entities = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        return list(set(self.filter_null(entities)))

    def parse_data_properties(self):
        data_properties = [el for el in self.get_child_node(self.root, 'owl:DatatypeProperty')]
        fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_data_properties = [el for el in fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        inv_fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_data_properties = [el for el in inv_fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        return data_properties + fn_data_properties + inv_fn_data_properties
        
    def parse_object_properties(self):
        obj_properties = [el for el in self.get_child_node(self.root, 'owl:ObjectProperty')]
        fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_obj_properties = [el for el in fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        inv_fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_obj_properties = [el for el in inv_fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        return obj_properties + fn_obj_properties + inv_fn_obj_properties
    
    def get_object_properties(self):
        obj_props = [self.extract_ID(el) for el in self.object_properties]
        return list(set(self.filter_null(obj_props)))
    
    def get_data_properties(self):
        data_props = [self.extract_ID(el) for el in self.data_properties]
        return list(set(self.filter_null(data_props)))




In [3]:
# Extracting USE embeddings

ontologies_in_alignment = [l.split(".")[0].split("-") for l in os.listdir("reference-alignment/")]

def extractUSEEmbeddings(words):
    try:
        embed = hub.KerasLayer(USE_folder)
    except Exception as e:
        !mkdir $USE_folder
        !curl -L "https://tfhub.dev/google/universal-sentence-encoder-large/5?tf-hub-format=compressed" | tar -zxvC $USE_folder
        embed = hub.KerasLayer(USE_folder)
        pass
    word_embeddings = embed(words)
    return word_embeddings.numpy()

def cos_sim(a,b):
    return 1 - spatial.distance.cosine(a, b)

def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0).lower() for m in matches]

def parse(word):
    return flatten([el.split("_") for el in camel_case_split(word)])
    

extracted_elems = []

for ont_name in list(set(flatten(ontologies_in_alignment))):
    ont = Ontology("conference_ontologies/" + ont_name + ".owl")
    entities = ont.get_entities()
    props = ont.get_object_properties() + ont.get_data_properties()
    triples = list(set(flatten(ont.get_triples())))
    extracted_elems.extend([ont_name + "#" + elem for elem in entities + props + triples])

extracted_elems = list(set(extracted_elems))

inp = [" ".join(parse(word.split("#")[1])) for word in extracted_elems]
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\S+")
X = vectorizer.fit_transform(inp)
word2idx_tfidf = {word: i for (i, word)  in enumerate(vectorizer.get_feature_names())}
entity2idx_tfidf = {word.split("#")[1]: i for (i, word)  in enumerate(extracted_elems)}


print ("Total number of extracted unique classes and properties from entire RA set: ", len(extracted_elems))

inp = ["<UNK>"] + inp
extracted_elems = ["<UNK>"] + extracted_elems

embeds = extractUSEEmbeddings(inp)
embeddings = dict(zip(extracted_elems, embeds))    

Total number of extracted unique classes and properties from entire RA set:  834


In [4]:
# Type storage

types_dict = {}

def get_tfidf_score(word, phrase):
    return np.sum([X[entity2idx_tfidf[phrase]][:,word2idx_tfidf[word]][0,0] for word in parse(phrase)])
    
for ont_name in list(set(flatten(ontologies_in_alignment))):
    ont = Ontology("conference_ontologies/" + ont_name + ".owl")
    
    entities = ont.get_entities()
    props = ont.get_object_properties() + ont.get_data_properties()

    for entity in entities:
        types_dict[entity] = {"type": "entity"}
    for prop in props:
        types_dict[prop] = {"type": "property"}


In [5]:
# Combinatorial mapping generation

all_mappings = []
for l in ontologies_in_alignment:
    ont1 = Ontology("conference_ontologies/" + l[0] + ".owl")
    ont2 = Ontology("conference_ontologies/" + l[1] + ".owl")
    
    ent1 = ont1.get_entities()
    ent2 = ont2.get_entities()
    
    obj1 = ont1.get_object_properties()
    obj2 = ont2.get_object_properties()
    
    data1 = ont1.get_data_properties()
    data2 = ont2.get_data_properties()
    
    mappings = list(itertools.product(ent1, ent2)) + list(itertools.product(obj1, obj2)) + list(itertools.product(data1, data2))
    
    all_mappings.extend([(l[0] + "#" + el[0], l[1] + "#" + el[1]) for el in mappings])
    

In [6]:
gt_mappings = [tuple([elem.split("/")[-1] for elem in el]) for el in reference_alignments]

data = {}
for mapping in all_mappings:
    if mapping in gt_mappings:
        data[(mapping[0], mapping[1])] = True
    else:
        data[(mapping[0], mapping[1])] = False


In [7]:

def greedy_matching():
    global batch_size, test_data_t, test_data_f, model, optimizer, emb_indexer_inv, gt_mappings, all_metrics
    all_results = OrderedDict()
    with torch.no_grad():
        all_pred = []
        batch_size = min(batch_size, len(test_data_t))
        num_batches = int(ceil(len(test_data_t)/batch_size))
        batch_size_f = int(ceil(len(test_data_f)/num_batches))
        
        np.random.shuffle(test_data_t)
        np.random.shuffle(test_data_f)

        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx+1) * batch_size

            batch_start_f = batch_idx * batch_size_f
            batch_end_f = (batch_idx+1) * batch_size_f
            
            
            
            pos_elems = np.array(test_data_t)[batch_start:batch_end]
            neg_elems = np.array(test_data_f)[batch_start_f:batch_end_f]
            optimizer.zero_grad()

            inputs = np.array([generate_data(elem) for elem in list(pos_elems) + list(neg_elems)])
            
            targets = np.array([1 for i in range(len(pos_elems))] + [0 for i in range(len(neg_elems))])
            
            print (inputs)
            
            indices = np.random.permutation(inputs.shape[0])
            inputs, targets = inputs[indices].transpose(1,0,2), targets[indices]
#             inputs = inputs.transpose(1,0,2)
            inputs_elem = inputs.copy()
            
#             print ("1", inputs_elem)
            
            nonzero_elems = np.count_nonzero(inputs, axis=-1)
            indices = np.flip(np.argsort(nonzero_elems, axis=-1), axis=-1)
            seq_lens = np.flip(np.sort(nonzero_elems, axis=-1), axis=-1)
#             print (seq_lens)
            inputs = np.stack((inputs[0][[indices[0]]], inputs[1][[indices[1]]]), axis=0)
            
            d1 = {elem:i for i,elem in enumerate(indices[0])}
            d2 = {elem:i for i,elem in enumerate(indices[1])}
            rev_indices = np.stack(([d1[k] for k in range(inputs_elem.shape[1])], [d2[k] for k in range(inputs_elem.shape[1])]))
#             print ("2", rev_indices)
#             print ("3", indices)
            rev_indices = torch.LongTensor(rev_indices)
            inputs = torch.LongTensor(inputs)
            seq_lens = torch.LongTensor(seq_lens.copy())
            targets = torch.DoubleTensor(targets)

            outputs = model(inputs, seq_lens, rev_indices)
            outputs /= torch.sum(outputs, dim=1).view(-1, 1)
#             write (("Outputs Finally: ", str([str(s) for s in outputs])))
            outputs = [(1-el[1].item()) for el in outputs]
    
#             return
#             print ("2", inputs)
#             print ("3", seq_lens)
#             print ("4", rev_indices)
            
            targets = [True if el.item() else False for el in targets]
#             print (inputs)
            for idx, pred_elem in enumerate(outputs):
                ent1 = emb_indexer_inv[inputs_elem[0][idx][0]]
                ent2 = emb_indexer_inv[inputs_elem[1][idx][0]]
                if (ent1, ent2) in all_results:
                    print ("Error: ", ent1, ent2, "already present")
                all_results[(ent1, ent2)] = (pred_elem, targets[idx])
        
        all_results = OrderedDict(sorted(all_results.items(), key=lambda x: x[0], reverse=True))
        filtered_results = dict()
        
        entities_to_assign = set([el[0] for el in list(all_results.keys())])
        for pair in all_results:
            if pair[0] in entities_to_assign:
                filtered_results[pair] = all_results[pair]
                entities_to_assign.remove(pair[0])
                
        entities_to_assign = set([el[1] for el in list(all_results.keys())])
        for pair in all_results:
            if pair[1] in entities_to_assign:
                filtered_results[pair] = all_results[pair]
                entities_to_assign.remove(pair[1])        

        filtered_results = OrderedDict(sorted(filtered_results.items(), key=lambda x: x[1][0], reverse=True))
        
        optimum_metrics, opt_threshold = [-1000 for i in range(5)], -1000
        low_threshold = np.min([el[0] for el in all_results.values()]) - 0.01
        high_threshold = np.max([el[0] for el in all_results.values()]) + 0.01
        for j,threshold in enumerate(np.arange(low_threshold, high_threshold, 0.01)):
            res = []
            for i,key in enumerate(all_results):
                if all_results[key][0] > threshold:
                    res.append(key)
            fn_list = [key for key in gt_mappings if key not in set(res) and not is_valid(test_onto, key)]
            fp_list = [elem for elem in res if not all_results[elem][1]]
            tp_list = [elem for elem in res if all_results[elem][1]]
            
            tp, fn, fp = len(tp_list), len(fn_list), len(fp_list)
            
            
            try:
                precision = tp/(tp+fp)
                recall = tp/(tp+fn)
                f1score = 2 * precision * recall / (precision + recall)
                f2score = 5 * precision * recall / (4 * precision + recall)
                f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
            except Exception as e:
                print (e)
                continue
            print ("Threshold: ", threshold, precision, recall, f1score, f2score, f0_5score)

            if f1score > optimum_metrics[2]:
                optimum_metrics = [precision, recall, f1score, f2score, f0_5score]
                opt_threshold = threshold
        
        print ("Precision: {} Recall: {} F1-Score: {} F2-Score: {} F0.5-Score: {}".format(*optimum_metrics))
        all_metrics.append((opt_threshold, optimum_metrics))
    return all_results


In [8]:
ind_test, inp_test1, inp_test2 = None, None, None

def write(elem):
    f = open("Logs", "a+")
    if type(elem) == list or type(elem) == tuple:
        string = str("\n".join([str(s) for s in elem]))
    else:
        string = str(elem)
    f.write("\n"+string)
    f.close()
    
inputs3, results3 = None, None

class SiameseNetwork(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, num_layers):
        super().__init__() 
        
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.name_embedding = nn.Embedding(len(embeddings), self.embedding_dim)
        self.name_embedding.load_state_dict({'weight': torch.from_numpy(np.array(emb_vals))})

        self.dropout = dropout
        
        self.lstm = nn.LSTM(self.embedding_dim, self.hidden_dim, self.num_layers, batch_first=True)
        self.cosine_sim_layer = nn.CosineSimilarity(dim=1)
        self.layer1 = nn.Bilinear(self.hidden_dim, self.hidden_dim, 2)

    def forward(self, inputs, seq_lens, rev_indices):
        results = []
        for i in range(2):
            x = self.name_embedding(inputs[i])
#             print ("Embeddings", x)
            packed_inp = pack_padded_sequence(x, seq_lens[i].numpy(), batch_first=True)
            op, (ht, ct) = self.lstm(x)
            x = ht[2*(self.num_layers-1):].permute(1,0,2)
            x = x[rev_indices[i],:,:]
            results.append(x.reshape(-1, self.hidden_dim))
        global inputs3, results3
        results3 = results
        inputs3 = inputs
        x = self.layer1(results[0], results[1])
        return x

In [57]:

emb_indexer = {word: i for i, word in enumerate(list(embeddings.keys()))}
emb_indexer_inv = {i: word for i, word in enumerate(list(embeddings.keys()))}
emb_vals = list(embeddings.values())


def get_one_hop_neighbours(ont, K=1):
    ont_obj = Ontology("conference_ontologies/" + ont + ".owl")
    triples = ont_obj.get_triples()
    entities = [(a,b) for (a,b,c) in triples]
    neighbours_dict = {elem: [elem] for elem in list(set(flatten(entities)))}
    for e1, e2 in entities:
        neighbours_dict[e1].append(e2)
        neighbours_dict[e2].append(e1)
    
    prop_triples = ont_obj.get_triples(subclass_of=False)
    neighbours_dict_props = {c: [c] for a,b,c in prop_triples}
    for e1, e2, p in prop_triples:
        neighbours_dict_props[p].extend([e1, e2])

    neighbours_dict = {**neighbours_dict, **neighbours_dict_props}
    
    for elem in ont_obj.get_entities() + ont_obj.get_object_properties() + ont_obj.get_data_properties():
        if elem not in neighbours_dict:
            neighbours_dict[elem] = [elem]

    neighbours_dict = {el: neighbours_dict[el][:1] + sorted(list(set(neighbours_dict[el][1:])))
                       for el in neighbours_dict}
    neighbours_dict = {el: neighbours_dict[el][:15] for el in neighbours_dict}
    neighbours_dict = {ont + "#" + el: [ont + "#" + e for e in neighbours_dict[el]] for el in neighbours_dict}
    return neighbours_dict

def is_valid(test_onto, key):
    return tuple([el.split("#")[0] for el in key]) not in test_onto

def generate_data(elem_tuple):
    op = np.array([[emb_indexer[el] for el in neighbours_dicts[elem.split("#")[0]][elem]] for elem in elem_tuple])
    return op

def generate_input(elems, target):
    inputs = np.array([generate_data(elem) for elem in list(elems)])
    targets = np.array([target for i in range(len(elems))])
    return inputs, targets

neighbours_dicts = {ont: get_one_hop_neighbours(ont) for ont in list(set(flatten(ontologies_in_alignment)))}
max_neighbours = np.max(flatten([[len(el[e]) for e in el] for el in neighbours_dicts.values()]))
neighbours_lens = {ont: {key: len(neighbours_dicts[ont][key]) for key in neighbours_dicts[ont]}
                   for ont in neighbours_dicts}
neighbours_dicts = {ont: {key: neighbours_dicts[ont][key] + ["<UNK>" for i in range(max_neighbours -len(neighbours_dicts[ont][key]))]
              for key in neighbours_dicts[ont]} for ont in neighbours_dicts}

# data_items = data.items()
# np.random.shuffle(list(data_items))
# data = OrderedDict(data_items)

data = OrderedDict(sorted(data.items(), key=lambda x: x[0]))

print ("Number of entities:", len(data))
# all_ont_pairs = list(set([tuple([el.split("#")[0] for el in l]) for l in data.keys()]))

all_metrics = []

for i in list(range(0, len(all_ont_pairs), 3)):
    
    test_onto = all_ont_pairs[i:i+3]
    
    train_data = {elem: data[elem] for elem in data if tuple([el.split("#")[0] for el in elem]) not in test_onto}
    test_data = {elem: data[elem] for elem in data if tuple([el.split("#")[0] for el in elem]) in test_onto}

    torch.set_default_dtype(torch.float64)
    
    train_test_split = 0.9

    train_data_t = [key for key in train_data if data[key]]
    train_data_f = [key for key in train_data if not data[key]][:len(train_data_t)]
#     [:int(0.1*(len(train_data) - len(train_data_t)) )]
#     np.random.shuffle(train_data_f)
    
    lr = 0.001
    num_epochs = 50
    weight_decay = 0.001
    batch_size = 10
    dropout = 0.3
    batch_size = min(batch_size, len(train_data_t))
    num_batches = int(ceil((len(train_data_t) + len(train_data_f))/batch_size))
    
    
    model = SiameseNetwork(512, 512, 1)

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    for epoch in range(num_epochs):
        inputs_pos, targets_pos = generate_input(train_data_t, 1)
        inputs_neg, targets_neg = generate_input(train_data_f, 0)
        
#         indices = np.random.permutation(len(inputs_pos) + len(inputs_neg))
        
#         inputs = np.array(list(inputs_pos) + list(inputs_neg))[indices]
#         targets = np.array(list(targets_pos) + list(targets_neg))[indices]

        inputs = np.array(list(inputs_pos) + list(inputs_neg))
        targets = np.array(list(targets_pos) + list(targets_neg))

        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx+1) * batch_size
            
            inp = inputs[batch_start:batch_end].transpose(1,0,2)
#             print (inputs[batch_start:batch_end].resha.shape)
            nonzero_elems = np.count_nonzero(inp, axis=-1)
            indices = np.flip(np.argsort(nonzero_elems, axis=-1), axis=-1)
            seq_lens = np.flip(np.sort(nonzero_elems, axis=-1), axis=-1)
            inp_elems = np.stack((inp[0][[indices[0]]], inp[1][[indices[1]]]), axis=0)
            
            inp_elems = torch.LongTensor(inp_elems)
            targ_elems = torch.DoubleTensor(targets[batch_start:batch_end])
            optimizer.zero_grad()
            
            d1 = {elem:i for i,elem in enumerate(indices[0])}
            d2 = {elem:i for i,elem in enumerate(indices[1])}
            rev_indices = np.stack(([d1[k] for k in range(inp.shape[1])], 
                                    [d2[k] for k in range(inp.shape[1])]))

            rev_indices = torch.LongTensor(rev_indices)
            seq_lens = torch.LongTensor(seq_lens.copy())
#             print (inp_elems)
            outputs = model(inp_elems, seq_lens, rev_indices)
#             print ("outputs", outputs)
#             print ("targets", targ_elems)
            loss = F.mse_loss(outputs, targ_elems)
            loss.backward()
#             break
            optimizer.step()

            if batch_idx%10 == 0:
                print ("Epoch: {} Idx: {} Loss: {}".format(epoch, batch_idx, loss.item()))

#         break
    model.eval()
#     break
    
    test_data_t = [key for key in test_data if data[key]]
    test_data_f = [key for key in test_data if not data[key]]
    
    res = greedy_matching()
    break
# print ("Final Results: ", np.mean([el[1] for el in all_metrics], axis=0))
# print ("Best threshold: ", all_metrics[np.argmax([el[1][2] for el in all_metrics])][0])

Number of entities: 122893


/home/vlead/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:112: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


Epoch: 0 Idx: 0 Loss: 2.087074380623987e-05
Epoch: 0 Idx: 10 Loss: 1.3472349999682434e-08
Epoch: 0 Idx: 20 Loss: 1.3424823947934334e-13
Epoch: 0 Idx: 30 Loss: 0.9999999784640442
Epoch: 0 Idx: 40 Loss: 0.9999999708097326
Epoch: 0 Idx: 50 Loss: 0.9055666863689836
Epoch: 1 Idx: 0 Loss: 5.877148812401375e-09
Epoch: 1 Idx: 10 Loss: 2.5111411006464773e-14
Epoch: 1 Idx: 20 Loss: 4.675262043629766e-16
Epoch: 1 Idx: 30 Loss: 0.9999999839126194
Epoch: 1 Idx: 40 Loss: 0.9999999937796236
Epoch: 1 Idx: 50 Loss: 0.9999994438889962
Epoch: 2 Idx: 0 Loss: 2.924882980596831e-13
Epoch: 2 Idx: 10 Loss: 1.247556219719615e-14
Epoch: 2 Idx: 20 Loss: 1.1764164775961952e-16
Epoch: 2 Idx: 30 Loss: 0.9999999152870386


KeyboardInterrupt: 

In [52]:

def djikstra(n, adj_matrix, start_node): 

    distances = [sys.maxsize] * n 
    path = [False] * n
    
    distances[start_node] = 0
    
    for node in range(n): 
        
        distances_dict = {elem: i for (i,elem) in enumerate(distances) if not path[i]}
        closest_node = distances_dict[min(list(distances_dict.keys()))]

        path[closest_node] = True
 
        for curr_node in range(n): 
            if adj_matrix[closest_node][curr_node] > 0 and not path[curr_node] and \
            distances[curr_node] > distances[closest_node] + adj_matrix[closest_node][curr_node]: 
                distances[curr_node] = distances[closest_node] + adj_matrix[closest_node][curr_node]

    return distances

all_triples = Ontology("conference_ontologies/conference.owl").get_triples()

entities = {entity:i for i,entity in enumerate(list(set(flatten([(el[0], el[1]) for el in all_triples]))))}
entities_inv = {entities[entity]:entity for entity in entities}

adj_mat = np.zeros((len(entities), len(entities)))

for (a,b,_) in all_triples:
    adj_mat[entities[a]][entities[b]] = 1
    adj_mat[entities[b]][entities[a]] = 1

src = entities["Conference_part"]
sorted([(entities_inv[i], entity) for i,entity in enumerate(djikstra(len(entities), adj_mat, src))], key=lambda x:x[1])

[('Conference_part', 0),
 ('Workshop', 1.0),
 ('string', 1.0),
 ('Tutorial', 1.0),
 ('Topic', 1.0),
 ('Track', 1.0),
 ('Track-workshop_chair', 1.0),
 ('Conference_volume', 1.0),
 ('Program_committee', 2.0),
 ('Person', 2.0),
 ('Conference', 2.0),
 ('Organizing_committee', 2.0),
 ('Steering_committee', 2.0),
 ('Important_dates', 2.0),
 ('Review_preference', 2.0),
 ('Conference_www', 2.0),
 ('Conference_contribution', 2.0),
 ('Committee', 2.0),
 ('Conference_proceedings', 2.0),
 ('Publisher', 2.0),
 ('Committee_member', 3.0),
 ('Submitted_contribution', 3.0),
 ('Written_contribution', 3.0),
 ('date', 3.0),
 ('Conference_contributor', 3.0),
 ('Poster', 3.0),
 ('int', 3.0),
 ('Co-chair', 3.0),
 ('Conference_document', 3.0),
 ('Conference_participant', 3.0),
 ('Chair', 3.0),
 ('Conference_applicant', 3.0),
 ('Reviewer', 3.0),
 ('Presentation', 3.0),
 ('Thing', 3.0),
 ('Review_expertise', 4.0),
 ('Invited_speaker', 4.0),
 ('Reviewed_contribution', 4.0),
 ('Invited_talk', 4.0),
 ('Registeered

In [46]:
class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return output

['Workshop',
 'string',
 'Tutorial',
 'Topic',
 'Track',
 'Track-workshop_chair',
 'Conference_volume']

In [14]:
def processPawsDataset(file):
    rows = [l.split("\t")[1:] for l in open(file, "r").read().split("\n")[1:-1]]
    data = []
    detokenizer = TreebankWordDetokenizer()
    for row in rows:
        a = detokenizer.detokenize(row[0].split())
        b = detokenizer.detokenize(row[1].split())
        c = float(row[2])
        data.append((a,b,c))
    return data

train_sents = processPawsDataset("paws/train.tsv")
val_sents = processPawsDataset("paws/dev_2k.tsv")
test_sents = processPawsDataset("paws/test_2k.tsv")

flatten = lambda l: [item for sublist in l for item in sublist]

all_sents = list(set(flatten([[el[0], el[1]] for el in train_sents + val_sents + test_sents])))

embeds = extractUSEEmbeddings(all_sents)
embeddings =  dict(zip(all_sents, embeds))

In [24]:

def greedy_matching():
    global batch_size, test_data_t, test_data_f, model, optimizer, emb_indexer_inv, gt_mappings, all_metrics
    all_results = OrderedDict()
    with torch.no_grad():
        all_pred = []
        batch_size = min(batch_size, len(test_data_t))
        num_batches = int(ceil(len(test_data_t)/batch_size))
        batch_size_f = int(ceil(len(test_data_f)/num_batches))
        
        print ("F batch size: ", batch_size_f)
        np.random.shuffle(test_data_t)
        np.random.shuffle(test_data_f)

        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx+1) * batch_size

            batch_start_f = batch_idx * batch_size_f
            batch_end_f = (batch_idx+1) * batch_size_f
            
            pos_elems = np.array(test_data_t)[batch_start:batch_end]
            neg_elems = np.array(test_data_f)[batch_start_f:batch_end_f]
            optimizer.zero_grad()

            inputs = np.array([generate_data(elem) for elem in list(pos_elems) + list(neg_elems)])
            print ("Inputs len: ", len(inputs))
            targets = np.array([1 for i in range(len(pos_elems))] + [0 for i in range(len(neg_elems))])

            indices = np.random.permutation(inputs.shape[0])
            inputs, targets = inputs[indices], targets[indices]
            inputs = torch.LongTensor(list(zip(*inputs)))
            targets = torch.LongTensor(targets)

            outputs = model(inputs, 1)
            outputs = [el.item() for el in outputs]
#             print ([(el, targets[i]) for i,el in enumerate(outputs) if targets[i]])
            print ("Targets: ", targets)
#             write (("Outputs initially: ", str([str(s) for s in outputs])))
#             outputs /= torch.sum(outputs, dim=1).view(-1, 1)
#             write (("Outputs Finally: ", str([str(s) for s in outputs])))
#             outputs = [el[1].item() for el in outputs]
            
            
            targets = [True if el.item() else False for el in targets]

            for idx, pred_elem in enumerate(outputs):
                ent1 = emb_indexer_inv[inputs.numpy()[0][idx]]
                ent2 = emb_indexer_inv[inputs.numpy()[1][idx]]
                if (ent1, ent2) in all_results:
                    print ("Error: ", ent1, ent2, "already present")
                all_results[(ent1, ent2)] = (pred_elem, targets[idx])
        
#         all_results = OrderedDict(sorted(all_results.items(), key=lambda x: x[0], reverse=True))
#         filtered_results = dict()
#         entities_to_assign = set([el[0] for el in list(all_results.keys())])
#         for pair in all_results:
#             if pair[0] in entities_to_assign:
#                 filtered_results[pair] = all_results[pair]
#                 entities_to_assign.remove(pair[0])
#         filtered_results = OrderedDict(sorted(filtered_results.items(), key=lambda x: x[1][0], reverse=True))
#         min_val = np.min([el[0] for el in list(all_results.values())])
#         max_val = np.max([el[0] for el in list(all_results.values())])
#         normalized_results = {}
#         for key,val in all_results.items():
#             tmp = (np.array(val[0]) - min_val) / (max_val - min_val)
#             normalized_results[key] = (tmp[1], val[1])
        
        optimum_metrics, opt_threshold = [-1000 for i in range(5)], -1000
        low_threshold = np.min([el[0] for el in all_results.values()]) - 0.01
        high_threshold = np.max([el[0] for el in all_results.values()]) + 0.01
        print ("Low:", low_threshold, "High:", high_threshold)
        for j,threshold in enumerate(np.arange(low_threshold, high_threshold, 0.01)):
            res = []
            for i,key in enumerate(all_results):
                if all_results[key][0] > threshold:
                    res.append(key)

            fn_list = [key for key in gt_mappings if key not in set(res) and not is_valid(test_onto, key)]
            fp_list = [elem for elem in res if not all_results[elem][1]]
            tp_list = [elem for elem in res if all_results[elem][1]]
            
            tp, fn, fp = len(tp_list), len(fn_list), len(fp_list)
            
            
            try:
                precision = tp/(tp+fp)
                recall = tp/(tp+fn)
                f1score = 2 * precision * recall / (precision + recall)
                f2score = 5 * precision * recall / (4 * precision + recall)
                f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
            except Exception as e:
                print (e)
                continue
            print ("Threshold: ", threshold, precision, recall, f1score, f2score, f0_5score)

            if f1score > optimum_metrics[2]:
                optimum_metrics = [precision, recall, f1score, f2score, f0_5score]
                opt_threshold = threshold
        
        print ("Precision: {} Recall: {} F1-Score: {} F2-Score: {} F0.5-Score: {}".format(*optimum_metrics))
        all_metrics.append((opt_threshold, optimum_metrics))
        
    print ("Final Results: ", np.mean([el[1] for el in all_metrics], axis=0))
    print ("Best threshold: ", all_metrics[np.argmax([el[1][2] for el in all_metrics])][0])
    return all_results
    


In [16]:
train_data = {tuple(sent[:2]): sent[-1] for sent in train_sents + val_sents}
train_data_t = [key for key in train_data if train_data[key]]
train_data_f = [key for key in train_data if not train_data[key]]
len(train_data_t)/len(train_data)

0.4411638686557862

In [ ]:
train

In [31]:
def greedy_matching():
    global batch_size, test_data_t, test_data_f, model, optimizer, emb_indexer_inv, gt_mappings, all_metrics
    all_results = OrderedDict()
    with torch.no_grad():
        all_pred = []
        batch_size = min(batch_size, len(test_data_t))
        num_batches = int(ceil(len(test_data_t)/batch_size))
        batch_size_f = int(ceil(len(test_data_f)/num_batches))
        
        print ("F batch size: ", batch_size_f)
        np.random.shuffle(test_data_t)
        np.random.shuffle(test_data_f)

        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = (batch_idx+1) * batch_size

            batch_start_f = batch_idx * batch_size_f
            batch_end_f = (batch_idx+1) * batch_size_f
            
            pos_elems = np.array(test_data_t)[batch_start:batch_end]
            neg_elems = np.array(test_data_f)[batch_start_f:batch_end_f]
            optimizer.zero_grad()

            inputs = np.array([generate_data(elem) for elem in list(pos_elems) + list(neg_elems)])
            print ("Inputs len: ", len(inputs))
            targets = np.array([1 for i in range(len(pos_elems))] + [0 for i in range(len(neg_elems))])

            indices = np.random.permutation(inputs.shape[0])
            inputs, targets = inputs[indices], targets[indices]
            inputs = torch.LongTensor(list(zip(*inputs)))
            targets = torch.LongTensor(targets)

            outputs = model(inputs, 1)
            outputs = [el.item() for el in outputs]
#             print ([(el, targets[i]) for i,el in enumerate(outputs) if targets[i]])
            print ("Targets: ", targets)
#             write (("Outputs initially: ", str([str(s) for s in outputs])))
#             outputs /= torch.sum(outputs, dim=1).view(-1, 1)
#             write (("Outputs Finally: ", str([str(s) for s in outputs])))
#             outputs = [el[1].item() for el in outputs]
            
            
            targets = [True if el.item() else False for el in targets]

            for idx, pred_elem in enumerate(outputs):
                ent1 = emb_indexer_inv[inputs.numpy()[0][idx]]
                ent2 = emb_indexer_inv[inputs.numpy()[1][idx]]
                if (ent1, ent2) in all_results:
                    print ("Error: ", ent1, ent2, "already present")
                all_results[(ent1, ent2)] = (pred_elem, targets[idx])
        
#         all_results = OrderedDict(sorted(all_results.items(), key=lambda x: x[0], reverse=True))
#         filtered_results = dict()
#         entities_to_assign = set([el[0] for el in list(all_results.keys())])
#         for pair in all_results:
#             if pair[0] in entities_to_assign:
#                 filtered_results[pair] = all_results[pair]
#                 entities_to_assign.remove(pair[0])
#         filtered_results = OrderedDict(sorted(filtered_results.items(), key=lambda x: x[1][0], reverse=True))
#         min_val = np.min([el[0] for el in list(all_results.values())])
#         max_val = np.max([el[0] for el in list(all_results.values())])
#         normalized_results = {}
#         for key,val in all_results.items():
#             tmp = (np.array(val[0]) - min_val) / (max_val - min_val)
#             normalized_results[key] = (tmp[1], val[1])
        
        optimum_metrics, opt_threshold = [-1000 for i in range(5)], -1000
        low_threshold = np.min([el[0] for el in all_results.values()]) - 0.01
        high_threshold = np.max([el[0] for el in all_results.values()]) + 0.01
        print ("Low:", low_threshold, "High:", high_threshold)
        for j,threshold in enumerate(np.arange(low_threshold, high_threshold, 0.01)):
            res = []
            for i,key in enumerate(all_results):
                if all_results[key][0] > threshold:
                    res.append(key)

            fn_list = [key for key in test_data_t if key not in set(res)]
            fp_list = [elem for elem in res if not all_results[elem][1]]
            tp_list = [elem for elem in res if all_results[elem][1]]
            
            tp, fn, fp = len(tp_list), len(fn_list), len(fp_list)
            
            
            try:
                precision = tp/(tp+fp)
                recall = tp/(tp+fn)
                f1score = 2 * precision * recall / (precision + recall)
                f2score = 5 * precision * recall / (4 * precision + recall)
                f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
            except Exception as e:
                print (e)
                continue
            print ("Threshold: ", threshold, precision, recall, f1score, f2score, f0_5score)

            if f1score > optimum_metrics[2]:
                optimum_metrics = [precision, recall, f1score, f2score, f0_5score]
                opt_threshold = threshold
        
        print ("Precision: {} Recall: {} F1-Score: {} F2-Score: {} F0.5-Score: {}".format(*optimum_metrics))
        all_metrics.append((opt_threshold, optimum_metrics))
        
    print ("Final Results: ", np.mean([el[1] for el in all_metrics], axis=0))
    print ("Best threshold: ", all_metrics[np.argmax([el[1][2] for el in all_metrics])][0])
    return all_results
    


In [32]:
def write(elem):
    f = open("Logs", "a+")
    if type(elem) == list or type(elem) == tuple:
        string = str("\n".join([str(s) for s in elem]))
    else:
        string = str(elem)
    f.write("\n"+string)
    f.close()
    

class SiameseNetwork(nn.Module):
    def __init__(self):
        super().__init__()    
        self.name_embedding = nn.Embedding(len(embeddings), 512)
        self.name_embedding.load_state_dict({'weight': torch.from_numpy(np.array(emb_vals))})
        self.name_embedding.require_grad = False
        
        self.dropout = dropout
        self.cosine_sim_layer = nn.CosineSimilarity(dim=1)
        self.layer1 = nn.Bilinear(512, 512, 2)
        self.layer2 = nn.Linear(512, 10)

    def forward(self, inputs, epoch):
        results = []
        for i in range(2):
            x = self.name_embedding(inputs[i])
            x = self.layer2(x)
            x = F.dropout(x, p=0.3)
            results.append(x)
        x = self.cosine_sim_layer(results[0], results[1])
        return x


In [52]:
emb_indexer = {word: i for i, word in enumerate(list(embeddings.keys()))}
emb_indexer_inv = {i: word for i, word in enumerate(list(embeddings.keys()))}
emb_vals = list(embeddings.values())

def generate_data(elem_tuple):
    return np.array([emb_indexer[elem] for elem in elem_tuple])

def generate_input(elems, target):
    inputs = np.array([generate_data(elem) for elem in list(elems)])
    targets = np.array([target for i in range(len(elems))])
    return inputs, targets

train_data = {tuple(sent[:2]): sent[-1] for sent in train_sents + val_sents}
data_items = train_data.items()
np.random.shuffle(list(data_items))
train_data = OrderedDict(data_items)

test_data = {tuple(sent[:2]): sent[-1] for sent in test_sents}
data_items = test_data.items()
np.random.shuffle(list(data_items))
test_data = OrderedDict(data_items)

print ("Number of entities:", len(train_data))

all_metrics = []
    
torch.set_default_dtype(torch.float64)

train_data_t = [key for key in train_data if train_data[key]]
train_data_f = [key for key in train_data if not train_data[key]]
train_data_f = train_data_f[:int(len(train_data_t))]
np.random.shuffle(train_data_f)



lr = 0.001
num_epochs = 50
weight_decay = 0.001
batch_size = 10
dropout = 0.3
batch_size = min(batch_size, len(train_data_t))
num_batches = int(ceil(len(train_data_t)/batch_size))
batch_size_f = int(ceil(len(train_data_f)/num_batches))

model = SiameseNetwork()

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

for epoch in range(num_epochs):
    inputs_pos, targets_pos = generate_input(train_data_t, 1)
    inputs_neg, targets_neg = generate_input(train_data_f, 0)

#         indices = np.random.permutation(len(inputs_pos) + len(inputs_neg))

#         inputs = np.array(list(inputs_pos) + list(inputs_neg))[indices]
#         targets = np.array(list(targets_pos) + list(targets_neg))[indices]

    indices_pos = np.random.permutation(len(inputs_pos))
    indices_neg = np.random.permutation(len(inputs_neg))

    inputs_pos, targets_pos = inputs_pos[indices_pos], targets_pos[indices_pos]
    inputs_neg, targets_neg = inputs_neg[indices_neg], targets_neg[indices_neg]
#         inputs = np.array(list(inputs_pos) + list(inputs_neg))[indices]
#         targets = np.array(list(targets_pos) + list(targets_neg))[indices]


#         inputs = np.array(list(inputs_pos) + list(inputs_neg))
#         targets = np.array(list(targets_pos) + list(targets_neg))

    for batch_idx in range(num_batches):
        batch_start = batch_idx * batch_size
        batch_end = (batch_idx+1) * batch_size

        batch_start_f = batch_idx * batch_size_f
        batch_end_f = (batch_idx+1) * batch_size_f

        inputs = np.concatenate((inputs_pos[batch_start: batch_end], inputs_neg[batch_start_f: batch_end_f]))
        targets = np.concatenate((targets_pos[batch_start: batch_end], targets_neg[batch_start_f: batch_end_f]))
#             print (inputs.shape)
        inp_elems = torch.LongTensor(inputs.T)
        targ_elems = torch.DoubleTensor(targets)
#             print (targ_elems)
        optimizer.zero_grad()

        outputs = model(inp_elems, epoch)
#             print (outputs)
        loss = F.mse_loss(outputs, targ_elems)
        loss.backward()

        optimizer.step()

        if batch_idx%10 == 0:
#                 print ("Outupts: ", list(zip(outputs.detach().numpy(), targ_elems.detach().numpy())))
            print ("Epoch: {} Idx: {} Loss: {}".format(epoch, batch_idx, loss.item()))

model.eval()

test_data_t = [key for key in test_data if test_data[key]]
test_data_f = [key for key in test_data if not test_data[key]]

res = greedy_matching()


Number of entities: 51346
Epoch: 0 Idx: 0 Loss: 0.2735395993369103
Epoch: 0 Idx: 10 Loss: 0.23160435520057215
Epoch: 0 Idx: 20 Loss: 0.357919914284692
Epoch: 0 Idx: 30 Loss: 0.3599698330458351
Epoch: 0 Idx: 40 Loss: 0.24836438135361188
Epoch: 0 Idx: 50 Loss: 0.3741192436162126
Epoch: 0 Idx: 60 Loss: 0.3380235482209918
Epoch: 0 Idx: 70 Loss: 0.26079815436059783
Epoch: 0 Idx: 80 Loss: 0.35552498218410555
Epoch: 0 Idx: 90 Loss: 0.3234828984808469
Epoch: 0 Idx: 100 Loss: 0.2840198007957181
Epoch: 0 Idx: 110 Loss: 0.32607445302050575
Epoch: 0 Idx: 120 Loss: 0.2986414285658422
Epoch: 0 Idx: 130 Loss: 0.29289106386236696
Epoch: 0 Idx: 140 Loss: 0.28920401063745305
Epoch: 0 Idx: 150 Loss: 0.3239249451586222
Epoch: 0 Idx: 160 Loss: 0.35231266871266476
Epoch: 0 Idx: 170 Loss: 0.2626907907617582
Epoch: 0 Idx: 180 Loss: 0.24200401528447135
Epoch: 0 Idx: 190 Loss: 0.31528495271190604
Epoch: 0 Idx: 200 Loss: 0.2655964588574996
Epoch: 0 Idx: 210 Loss: 0.28975530154023144
Epoch: 0 Idx: 220 Loss: 0.195

Epoch: 0 Idx: 1870 Loss: 0.30162687832722396
Epoch: 0 Idx: 1880 Loss: 0.34611320503980486
Epoch: 0 Idx: 1890 Loss: 0.27868172676035036
Epoch: 0 Idx: 1900 Loss: 0.3022989455507262
Epoch: 0 Idx: 1910 Loss: 0.33729429081931667
Epoch: 0 Idx: 1920 Loss: 0.27835663885211587
Epoch: 0 Idx: 1930 Loss: 0.30521950784428364
Epoch: 0 Idx: 1940 Loss: 0.385605573040427
Epoch: 0 Idx: 1950 Loss: 0.3114990752003273
Epoch: 0 Idx: 1960 Loss: 0.279669396632204
Epoch: 0 Idx: 1970 Loss: 0.28172780951843446
Epoch: 0 Idx: 1980 Loss: 0.2684441097348117
Epoch: 0 Idx: 1990 Loss: 0.3003241961574251
Epoch: 0 Idx: 2000 Loss: 0.33011514303041806
Epoch: 0 Idx: 2010 Loss: 0.32078719189034266
Epoch: 0 Idx: 2020 Loss: 0.2930965377104705
Epoch: 0 Idx: 2030 Loss: 0.317229545059431
Epoch: 0 Idx: 2040 Loss: 0.30251735290224485
Epoch: 0 Idx: 2050 Loss: 0.2545362119318858
Epoch: 0 Idx: 2060 Loss: 0.3513793378197917
Epoch: 0 Idx: 2070 Loss: 0.2931267478332479
Epoch: 0 Idx: 2080 Loss: 0.3328703665889917
Epoch: 0 Idx: 2090 Loss: 

Epoch: 1 Idx: 1480 Loss: 0.2911697678718355
Epoch: 1 Idx: 1490 Loss: 0.29076988928241276
Epoch: 1 Idx: 1500 Loss: 0.3446946584310999
Epoch: 1 Idx: 1510 Loss: 0.23092575428985898
Epoch: 1 Idx: 1520 Loss: 0.2583962813830308
Epoch: 1 Idx: 1530 Loss: 0.358405009588145
Epoch: 1 Idx: 1540 Loss: 0.3496452357172136
Epoch: 1 Idx: 1550 Loss: 0.2395080902435327
Epoch: 1 Idx: 1560 Loss: 0.2996390756921582
Epoch: 1 Idx: 1570 Loss: 0.26426574650246665
Epoch: 1 Idx: 1580 Loss: 0.3677461218850005
Epoch: 1 Idx: 1590 Loss: 0.4286040208256864
Epoch: 1 Idx: 1600 Loss: 0.2438782731494052
Epoch: 1 Idx: 1610 Loss: 0.2681902706381693
Epoch: 1 Idx: 1620 Loss: 0.30591441064393043
Epoch: 1 Idx: 1630 Loss: 0.2715821168173701
Epoch: 1 Idx: 1640 Loss: 0.29935712668711556
Epoch: 1 Idx: 1650 Loss: 0.3060710978895831
Epoch: 1 Idx: 1660 Loss: 0.26687335102344156
Epoch: 1 Idx: 1670 Loss: 0.3219588769869401
Epoch: 1 Idx: 1680 Loss: 0.31338400783715836
Epoch: 1 Idx: 1690 Loss: 0.2934917576964882
Epoch: 1 Idx: 1700 Loss: 0

Epoch: 2 Idx: 1090 Loss: 0.2872658503730604
Epoch: 2 Idx: 1100 Loss: 0.301061276316003
Epoch: 2 Idx: 1110 Loss: 0.3593952432498191
Epoch: 2 Idx: 1120 Loss: 0.3190478514809728
Epoch: 2 Idx: 1130 Loss: 0.30208679864975296
Epoch: 2 Idx: 1140 Loss: 0.25006492525476764
Epoch: 2 Idx: 1150 Loss: 0.27368770116358243
Epoch: 2 Idx: 1160 Loss: 0.37610776319315575
Epoch: 2 Idx: 1170 Loss: 0.32369320328243006
Epoch: 2 Idx: 1180 Loss: 0.24088390022693265
Epoch: 2 Idx: 1190 Loss: 0.41447059227023075
Epoch: 2 Idx: 1200 Loss: 0.30964991018571497
Epoch: 2 Idx: 1210 Loss: 0.2948244686970307
Epoch: 2 Idx: 1220 Loss: 0.26401608139687605
Epoch: 2 Idx: 1230 Loss: 0.2039590781928983
Epoch: 2 Idx: 1240 Loss: 0.3295005492415396
Epoch: 2 Idx: 1250 Loss: 0.25433899278573063
Epoch: 2 Idx: 1260 Loss: 0.26201739354161135
Epoch: 2 Idx: 1270 Loss: 0.2186216462793089
Epoch: 2 Idx: 1280 Loss: 0.2855608151392675
Epoch: 2 Idx: 1290 Loss: 0.29007378789322397
Epoch: 2 Idx: 1300 Loss: 0.2307349366486297
Epoch: 2 Idx: 1310 Lo

Epoch: 3 Idx: 680 Loss: 0.34583015771043296
Epoch: 3 Idx: 690 Loss: 0.26112220819441134
Epoch: 3 Idx: 700 Loss: 0.32341150315485157
Epoch: 3 Idx: 710 Loss: 0.3126660583763109
Epoch: 3 Idx: 720 Loss: 0.32395633669964274
Epoch: 3 Idx: 730 Loss: 0.31966989473885954
Epoch: 3 Idx: 740 Loss: 0.2610622199869568
Epoch: 3 Idx: 750 Loss: 0.2536592339330974
Epoch: 3 Idx: 760 Loss: 0.264476989563883
Epoch: 3 Idx: 770 Loss: 0.2544200377143847
Epoch: 3 Idx: 780 Loss: 0.2549546146642317
Epoch: 3 Idx: 790 Loss: 0.28300643886634413
Epoch: 3 Idx: 800 Loss: 0.30073705112406895
Epoch: 3 Idx: 810 Loss: 0.29867442558113627
Epoch: 3 Idx: 820 Loss: 0.32526693261828077
Epoch: 3 Idx: 830 Loss: 0.25466799531151674
Epoch: 3 Idx: 840 Loss: 0.24529630229964422
Epoch: 3 Idx: 850 Loss: 0.2524733396983564
Epoch: 3 Idx: 860 Loss: 0.3407282148971225
Epoch: 3 Idx: 870 Loss: 0.3157047651520087
Epoch: 3 Idx: 880 Loss: 0.40830468804764647
Epoch: 3 Idx: 890 Loss: 0.29100200425205236
Epoch: 3 Idx: 900 Loss: 0.2967220092401895

...

Epoch: 10 Idx: 1490 Loss: 0.07828387422440554
Epoch: 10 Idx: 1500 Loss: 0.23169004243642913
Epoch: 10 Idx: 1510 Loss: 0.15685981692303083
Epoch: 10 Idx: 1520 Loss: 0.08044541228120315
Epoch: 10 Idx: 1530 Loss: 0.18748870986906155
Epoch: 10 Idx: 1540 Loss: 0.10105461433788385
Epoch: 10 Idx: 1550 Loss: 0.13326271074441504
Epoch: 10 Idx: 1560 Loss: 0.09370751944457957
Epoch: 10 Idx: 1570 Loss: 0.2055389795791472
Epoch: 10 Idx: 1580 Loss: 0.08949767852848882
Epoch: 10 Idx: 1590 Loss: 0.32408259133858464
Epoch: 10 Idx: 1600 Loss: 0.1337177619361492
Epoch: 10 Idx: 1610 Loss: 0.10680173363893983
Epoch: 10 Idx: 1620 Loss: 0.1154782376097598
Epoch: 10 Idx: 1630 Loss: 0.13591416513659593
Epoch: 10 Idx: 1640 Loss: 0.16465245893645378
Epoch: 10 Idx: 1650 Loss: 0.21785228088921596
Epoch: 10 Idx: 1660 Loss: 0.08974679918519263
Epoch: 10 Idx: 1670 Loss: 0.12761549401731204
Epoch: 10 Idx: 1680 Loss: 0.09982118816939159
Epoch: 10 Idx: 1690 Loss: 0.2062643712393685
Epoch: 10 Idx: 1700 Loss: 0.1655684337

Epoch: 11 Idx: 1040 Loss: 0.14898894205443855
Epoch: 11 Idx: 1050 Loss: 0.05500188706782081
Epoch: 11 Idx: 1060 Loss: 0.10788237266993421
Epoch: 11 Idx: 1070 Loss: 0.17605039443220077
Epoch: 11 Idx: 1080 Loss: 0.16240940852593574
Epoch: 11 Idx: 1090 Loss: 0.08135363996648373
Epoch: 11 Idx: 1100 Loss: 0.08838852355210715
Epoch: 11 Idx: 1110 Loss: 0.1324624207104595
Epoch: 11 Idx: 1120 Loss: 0.19144707000638497
Epoch: 11 Idx: 1130 Loss: 0.10985860498133529
Epoch: 11 Idx: 1140 Loss: 0.13256778168108346
Epoch: 11 Idx: 1150 Loss: 0.17376659578052744
Epoch: 11 Idx: 1160 Loss: 0.14987021529310235
Epoch: 11 Idx: 1170 Loss: 0.1334595894381176
Epoch: 11 Idx: 1180 Loss: 0.1681850864037831
Epoch: 11 Idx: 1190 Loss: 0.19704277529471248
Epoch: 11 Idx: 1200 Loss: 0.18937739952299035
Epoch: 11 Idx: 1210 Loss: 0.16891374369216064
Epoch: 11 Idx: 1220 Loss: 0.16874406594405353
Epoch: 11 Idx: 1230 Loss: 0.12894013739305948
Epoch: 11 Idx: 1240 Loss: 0.1611302029610037
Epoch: 11 Idx: 1250 Loss: 0.2309545392

Epoch: 12 Idx: 580 Loss: 0.14485522600375136
Epoch: 12 Idx: 590 Loss: 0.12823570628038078
Epoch: 12 Idx: 600 Loss: 0.185211259483695
Epoch: 12 Idx: 610 Loss: 0.07363093408068445
Epoch: 12 Idx: 620 Loss: 0.11255689517828449
Epoch: 12 Idx: 630 Loss: 0.21758889488167488
Epoch: 12 Idx: 640 Loss: 0.10091505843523885
Epoch: 12 Idx: 650 Loss: 0.10236543550797131
Epoch: 12 Idx: 660 Loss: 0.1424207023447803
Epoch: 12 Idx: 670 Loss: 0.09718573330877452
Epoch: 12 Idx: 680 Loss: 0.0995931756216759
Epoch: 12 Idx: 690 Loss: 0.09332392563971087
Epoch: 12 Idx: 700 Loss: 0.16006497951893137
Epoch: 12 Idx: 710 Loss: 0.10464253467526005
Epoch: 12 Idx: 720 Loss: 0.07640669948039569
Epoch: 12 Idx: 730 Loss: 0.10205944273040249
Epoch: 12 Idx: 740 Loss: 0.11778804201908659
Epoch: 12 Idx: 750 Loss: 0.15597578893738956
Epoch: 12 Idx: 760 Loss: 0.12733499609240603
Epoch: 12 Idx: 770 Loss: 0.1508505853724013
Epoch: 12 Idx: 780 Loss: 0.10463134774204819
Epoch: 12 Idx: 790 Loss: 0.09717933042445792
Epoch: 12 Idx: 

Epoch: 13 Idx: 120 Loss: 0.15064232182519402
Epoch: 13 Idx: 130 Loss: 0.11010179245249105
Epoch: 13 Idx: 140 Loss: 0.1833918942473295
Epoch: 13 Idx: 150 Loss: 0.13251814785105118
Epoch: 13 Idx: 160 Loss: 0.13798268890745252
Epoch: 13 Idx: 170 Loss: 0.05441479881344428
Epoch: 13 Idx: 180 Loss: 0.11595216407632476
Epoch: 13 Idx: 190 Loss: 0.1259717656700335
Epoch: 13 Idx: 200 Loss: 0.06330419697162361
Epoch: 13 Idx: 210 Loss: 0.0772865369516185
Epoch: 13 Idx: 220 Loss: 0.07672370395413082
Epoch: 13 Idx: 230 Loss: 0.09473286348664343
Epoch: 13 Idx: 240 Loss: 0.1298914490274202
Epoch: 13 Idx: 250 Loss: 0.10813287609033813
Epoch: 13 Idx: 260 Loss: 0.188550548891183
Epoch: 13 Idx: 270 Loss: 0.09647880606233111
Epoch: 13 Idx: 280 Loss: 0.16402134184067022
Epoch: 13 Idx: 290 Loss: 0.12257072226750307
Epoch: 13 Idx: 300 Loss: 0.12553476387461443
Epoch: 13 Idx: 310 Loss: 0.15963863134709605
Epoch: 13 Idx: 320 Loss: 0.13339893596408822
Epoch: 13 Idx: 330 Loss: 0.10304507764894226
Epoch: 13 Idx: 3

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [43]:
prefix = "dataset-sts/data/sts/semeval-sts/all/"
def process_sem_eval(file):
    sents = [l.split("\t") for l in open(prefix + file).read().split("\n")[:-1]]
    sents = [l for l in sents if l[0]]
    return sents

val_sents = process_sem_eval("2015.val.tsv")
train_sents = process_sem_eval("2015.train.tsv")
test_sents = process_sem_eval("2015.test.tsv")
all_sents = list(set(flatten([(l[1], l[2]) for l in train_sents + val_sents + test_sents])))
embeds = extractUSEEmbeddings(all_sents)
embeddings =  dict(zip(all_sents, embeds))



In [56]:
from sklearn.metrics import mean_squared_error

gt, pred = list(zip(*transform([(float(l[0]), cos_sim(embeddings[l[1]], embeddings[l[2]]))  for l in test_sents])))
mean_squared_error(gt, pred)

1.3655901006595916

In [64]:
t = torch.randn(1,3)
5*t, t

(tensor([[-1.1044,  2.9305,  3.9287]]), tensor([[-0.2209,  0.5861,  0.7857]]))